In [109]:
import pandas as pd
from transformers import pipeline
from bs4 import BeautifulSoup
import nltk
from nltk.stem.porter import *
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ThibJacq\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
nltk.download('stopwords', quiet=True)

True

In [122]:
df = pd.read_csv('./data/esa_csvs_raw.txt', sep='\t')

In [123]:
df.iloc[0]['csvText']

'[["Survey Date", "Aboriginal Communities", "Detail"], ["July 9 to 12, 2011", "Blueberry River First Nation <s>•</s>Halfway River First Nation <s>•</s>McLeod Lake Indian Band <s>•</s>North East Métis Association <s>•</s>", "Summer 2011 open water aquatic <s>•</s>assessments."], ["September 18 to October 10, 2011", "Blueberry River First Nation <s>•</s>Halfway River First Nation <s>•</s>McLeod Lake Indian Band <s>•</s>North East Métis Association <s>•</s>", "Fall 2011 open water aquatic assessments. <s>•</s>"], ["February 26 to March 12, 2012", "Blueberry River First Nation <s>•</s>Doig River First Nation <s>•</s>Halfway River First Nation <s>•</s>Kelly Lake Cree First Nation <s>•</s>McLeod Lake Indian Band <s>•</s>North East Métis Association <s>•</s>", "Winter 2012 aquatic habitat investigations. <s>•</s>"]]'

In [63]:
df_index = pd.read_csv('./data/ESA_website_ENG_2021_03_04_final.csv')

In [64]:
df_index.rename(columns={"Old CSV Filename": "csvFileName"}, inplace=True)

In [65]:
df_index.head()

,Index,Title,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,...,CSV Download URL,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,Missing CSV,CSV Filename,csvFileName
0,9134,TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABO...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrth...,14,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-3-summary-of-aquatics-field-wor...,1059614_14_lattice-v_1.csv
1,9135,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrth...,17,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_17_lattice-v_1.csv
2,9136,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrth...,18,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_18_lattice-v_1.csv
3,9137,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrth...,19,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_19_lattice-v_1.csv
4,9138,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrth...,20,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_20_lattice-v_1.csv


In [66]:
df_merged = df[df['csvFileName'].isin(list(df_index['csvFileName']))].reset_index(drop=True)

In [67]:
df_joined = pd.merge(df_index, df_merged, how='left', on='csvFileName')

In [68]:
df_joined.head()

,Index,Title,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,...,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,Missing CSV,CSV Filename,csvFileName,csvText
0,9134,TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABO...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,14,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-3-summary-of-aquatics-field-wor...,1059614_14_lattice-v_1.csv,Survey Date Aboriginal Communities Detail July...
1,9135,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,17,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_17_lattice-v_1.csv,Field Site No Name Legal Location UTM Coordina...
2,9136,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,18,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_18_lattice-v_1.csv,Field Site No Name Legal Location UTM Coordina...
3,9137,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,19,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_19_lattice-v_1.csv,Field Site No Name Legal Location UTM Coordina...
4,9138,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,20,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_20_lattice-v_1.csv,Field Site No Name Legal Location UTM Coordina...


In [69]:
len(df_joined)

28891

In [70]:
del df
del df_index
del df_merged

In [ ]:
df_joined.rename(columns={"csvText": "text"}, inplace=True)
df_joined['label'] = -1

In [ ]:
df_joined["text"] = df["Title"] + " " + df_joined["text"]
df_joined.drop(columns=["Title"], inplace=True)

In [124]:
df_joined['csvText'] = df_joined['csvText'].apply(lambda x: BeautifulSoup(x, "html.parser").get_text()) # remove html tags
df_joined['csvText'] = df_joined['csvText'].replace('[^a-zA-Z0-9 ]', '', regex=True) # remove all non-alpha-numeric characters
df_joined['csvText'] = df_joined['csvText'].replace(' +', ' ', regex=True) # remove all extra spaces in text

In [138]:
df['csvText'][1]

'Field Site No Name Legal Location UTM Coordinates NAD83 Zone 10 LatitudeLongitude DDMMSS Elevation m asl Fish Stream Classification 17 Fish and Wildlife Timing Window Low Risk 27 Riparian Management Area m 127 Open Water Mean Channel Morphology m 3 Fish Species Captured or Observed During Open Water Aquatic Assessment Previously Documented 4 Location of Nearest Known Fish Habitat 56 Beaver Activity Present YesNo Winter 2012 Aquatic Habitat Investigation Results Supplemental Fish Sampling Recommended for 2013 Comments 20112012 Aitken Creek Route CE WC0 Unnamed tributary to Stewart Creek SW 288020 W6M E 620133 N 6203041 55572764N 12143276W 873 m asl S4 or S6 TBD7 RRZ 0 RMZ 30 or 20 Bankfull 11 Wetted Dry Depth Dry No sampling Dry no fish previously documented Approximately 3 km downstream in Stewart Creek No Winter habitat investigations were not conducted due to dry conditions in fall 2011 Second season of fish sampling spring or summer WPT10 Unnamed tributary to the Pine River SE 6812

In [71]:
df_joined.iloc[0].csvText

'Survey Date Aboriginal Communities Detail July 9 to 12 2011 Blueberry River First Nation Halfway River First Nation McLeod Lake Indian Band North East Mtis Association Summer 2011 open water aquatic assessments September 18 to October 10 2011 Blueberry River First Nation Halfway River First Nation McLeod Lake Indian Band North East Mtis Association Fall 2011 open water aquatic assessments February 26 to March 12 2012 Blueberry River First Nation Doig River First Nation Halfway River First Nation Kelly Lake Cree First Nation McLeod Lake Indian Band North East Mtis Association Winter 2012 aquatic habitat investigations '

In [47]:
df_joined.to_csv('./data/esa_index_with_table_text_no_labels.csv')

In [48]:
df_joined = pd.read_csv('./data/esa_index_with_table_text_no_labels.csv')

In [125]:
for row in df.itertuples(index=True):
    tokenized_table_text = word_tokenize(df['csvText'][row.Index])
    if len(tokenized_table_text) > 256:
        df['csvText'][row.Index] = TreebankWordDetokenizer().detokenize(tokenized_table_text[0:256])
        
df.head()

,csvFileName,csvText
0,1059614_14_lattice-v_1.csv,Survey Date Aboriginal Communities Detail July...
1,1059614_17_lattice-v_1.csv,Field Site No Name Legal Location UTM Coordina...
2,1059614_18_lattice-v_1.csv,Field Site No Name Legal Location UTM Coordina...
3,1059614_19_lattice-v_1.csv,Field Site No Name Legal Location UTM Coordina...
4,1059614_20_lattice-v_1.csv,Field Site No Name Legal Location UTM Coordina...


In [134]:
for row in df_joined.iloc[0:100].itertuples():
    tokenized_table_text = word_tokenize(df_joined['csvText'][row.Index])
    assert len(tokenized_table_text) <= 256, "You have at least one table with more than 256 tokens."

In [ ]:
df_joined.to_csv('./data/esa_index_with_table_text_no_labels_256_max_tokens.csv')